## Imports

In [48]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage import transform
from tqdm import trange, tqdm
import os

## Get Data

In [ ]:
input_dir = './cancer_data/inputs/'
output_dir = './cancer_data/outputs/'

files = os.listdir(input_dir)
train_images, train_label, test_images, test_label = [], [], [], []
for f in tqdm(files):
    if 'train' in f:
        train_images.append(transform.resize(imread(input_dir + f), (512,512,3), mode='constant'))
        train_label.append(transform.resize(imread(output_dir + f), (512,512), mode='constant'))
    else:
        test_images.append(transform.resize(imread(input_dir + f), (512,512,3), mode='constant'))
        test_label.append(transform.resize(imread(output_dir + f), (512,512), mode='constant'))

#whiten the data
train_images = train_images - np.mean( train_images )
train_images = train_images / np.sqrt( np.var( train_images ) )

test_images = test_images - np.mean( test_images )
test_images = test_images / np.sqrt( np.var( test_images ) )

100%|██████████| 1517/1517 [05:34<00:00,  4.40it/s]


In [ ]:
train_images.shape

## Helper Functions

In [6]:
def conv( x, filter_size=3, stride=2, num_filters=64, is_output=False, name="conv" ):
    filter_height, filter_width = filter_size, filter_size
    in_channels = x.get_shape().as_list()[-1]
    out_channels = num_filters
    
    with tf.variable_scope(name) as scope:
        W = tf.get_variable("{}_W".format(name), shape=[filter_height, filter_width, in_channels, out_channels],
                            initializer = tf.contrib.layers.variance_scaling_initializer())
        b = tf.get_variable("{}_b".format(name), shape=[out_channels],
                            initializer = tf.contrib.layers.variance_scaling_initializer())
        conv = tf.nn.conv2d(x, W, [1, stride, stride, 1], padding="SAME")
        out = tf.nn.bias_add(conv, b)
        
        if not is_output:
            out = tf.nn.relu(out)
            
    return out

def convt( x, out_shape, filter_size=8, stride=2, is_output=False, name="convt" ):
    filter_height, filter_width = filter_size, filter_size
    in_channels = x.get_shape().as_list()[-1]

    with tf.variable_scope( name ):
        W = tf.get_variable( "{}_W".format(name), shape=[filter_height, filter_width, out_shape[-1], in_channels],
                            initializer = tf.contrib.layers.variance_scaling_initializer() )
        b = tf.get_variable( "{}_b".format(name), shape=[out_shape[-1]],
                            initializer = tf.contrib.layers.variance_scaling_initializer() )
        conv = tf.nn.conv2d_transpose( x, W, out_shape, [1, stride, stride, 1], padding="SAME" )
        out = tf.nn.bias_add( conv, b )
        if not is_output:
            out =  tf.nn.relu(out)

    return out

def fc( x, out_size=50, is_output=False, name="fc" ):
    in_size = x.get_shape().as_list()[0]
    
    with tf.variable_scope(name) as scope:
        W = tf.get_variable("{}_W".format(name), shape=[out_size, in_size], initializer = tf.contrib.layers.variance_scaling_initializer())
        b = tf.get_variable("{}_b".format(name), shape=[out_size, 1], initializer = tf.contrib.layers.variance_scaling_initializer())
        
        out = tf.matmul(W, x)
        out = out + b
        
        if not is_output:
            out = tf.nn.relu(out)
    return out

## Network Topology

In [ ]:
tf.reset_default_graph()

input_data = tf.placeholder(tf.float32, [1,512,512,3])
output_data = tf.placeholder(tf.float32, [1, 512, 512, 2])

h0 = conv(input_data, stride=1, name="h0") #[1, 32, 32, 64]
h1 = conv(h0, stride=2, name="h1") #[1, 16, 16, 64]
h2 = conv(h1, num_filters=32, name="h2") #[1, 8, 8, 32]
flat = tf.reshape(h2, [1*8*8*32, 1]) #[1*8*8*32,1]
# fc0 = fc(flat, name="fc0") #[50,1]
logits= fc(flat, out_size = 10, is_output=True, name="output") # [10,1]


with tf.name_scope( "loss_function" ) as scope:
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=tf.transpose(logits), labels=y_true)[0]

with tf.name_scope( "accuracy" ) as scope:
    correct_prediction = tf.equal( y_true, tf.argmax(logits,0) )
    accuracy = tf.reduce_mean( tf.cast(correct_prediction, tf.float32) )

train_step = tf.train.AdamOptimizer( 0.001 ).minimize( cross_entropy )

## Training

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

train_writer = tf.summary.FileWriter( "./tflogs", sess.graph )
 
tf.summary.scalar('cross_entropy', cross_entropy)
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()

tmp_loss = 0.0
accuracy_li = []
loss_li = []
for i in range( 100000 ):
    ind_train = np.random.randint(len(train_features))

    image_train = train_features[ind_train]
    label_train = train_labels[ind_train]

    acc_test = 0
    
    _, loss = sess.run( [train_step, cross_entropy], feed_dict={input_data: image_train.reshape([1,32,32,3]), y_true: label_train})
    tmp_loss += loss
    
    if i % 1000 == 0 and i != 0:
        for k in range(len(test_features)):
            acc_test += sess.run( accuracy, feed_dict={input_data: test_features[k].reshape([1,32,32,3]), y_true: test_labels[k]})

    if i % 1000 == 0 and i != 0:
        print( "i:{} test_acc:{} loss:{}".format( i, acc_test/len(test_features), tmp_loss ) )
        accuracy_li.append(acc_test/len(test_features))
        loss_li.append(tmp_loss)
        tmp_loss = 0.0

#saver.save( sess, './tf_logs/model.ckpt' )
train_writer.close()

## Plots

In [ ]:
plt.plot(accuracy_li)
plt.title("Accuracy")
plt.xlabel("in thousands of iterations")
plt.ylabel("% correct")
plt.show()

In [ ]:
plt.plot(loss_li)
plt.title("Loss")
plt.xlabel("in thousands of iterations")
plt.ylabel("average loss for 1000 iterations")
plt.show()

## Graph